In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [2]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [3]:
df = pd.read_csv('../datasets/drug_consumption.csv')

In [4]:
df.head()

,ID,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,...,Ecstasy,Heroin,Ketamine,Legalh,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,...,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,...,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,...,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,...,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0


In [5]:
target_columns = ['Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms','Nicotine', 'Semer', 'VSA']

In [6]:
for col in target_columns:
    df[col] = pd.to_numeric(df[col].str.replace('CL', '', regex=False), errors='coerce')

In [7]:
df.head()

,ID,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,...,Ecstasy,Heroin,Ketamine,Legalh,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,...,0,0,0,0,0,0,0,2,0,0
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,...,4,0,2,0,2,3,0,4,0,0
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,...,0,0,0,0,0,0,1,0,0,0
3,4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,...,0,0,2,0,0,0,0,2,0,0
4,5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,...,1,0,0,1,0,0,2,2,0,0


In [8]:
bdf = BaseDatasetTransform(df,target=['Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms',
       'Nicotine', 'Semer', 'VSA'])

In [9]:
df,categorical_features = bdf.fit_transform()


Обнаружены колонки с именем "id": ['ID']
Колонка "ID" удалена.
Series([], dtype: int64)

В колонке(ах) ['Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms', 'Nicotine', 'Semer', 'VSA'] нет пропущенных значений
-------------------------------------------
В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index(['Alcohol', 'Amphet', 'Amyl', 'Benzos', 'Caff', 'Cannabis', 'Choc',
       'Coke', 'Crack'],
      dtype='object')

Числовые колонки:
Index(['Age', 'Gender', 'Education', 'Country', 'Ethnicity', 'Nscore',
       'Escore', 'Oscore', 'Ascore', 'Cscore', 'Impulsive', 'SS', 'Ecstasy',
       'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms', 'Nicotine',
       'Semer', 'VSA'],
      dtype='object')


In [10]:
X_train, X_test, y_train, y_test  = bdf.get_train_test_split()

Количество значений целевой переменной по категориям:
Количество значений комбинаций целевых переменных по категориям:
0_0_0_0_0_0_0_0_0_0    291
0_0_0_0_0_0_0_1_0_0     91
0_0_0_0_0_0_0_6_0_0     90
0_0_0_0_0_0_0_2_0_0     82
0_0_0_0_0_0_0_3_0_0     34
                      ... 
3_0_0_0_0_0_2_6_0_0      1
3_4_4_0_3_2_5_6_0_1      1
2_0_0_0_3_0_5_3_0_2      1
0_0_0_2_1_0_3_0_0_0      1
3_0_0_3_3_0_3_6_0_2      1
Name: count, Length: 1107, dtype: int64
Следует ли выполнить стратифицированное раздеение на обучающую и тестовую выборку? y/n
Разделение датасета выполнено успешно


In [11]:
from sklearn.multioutput import MultiOutputClassifier


lgbm_model = MultiOutputClassifier(lgb.LGBMClassifier(verbose=-1))

columns_to_delete = []  

unique_classes = len(set(y_train))
print(unique_classes)

model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    #objective='binary',  
    #metric='binary_error' 
)

10


In [12]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics,base_multitarget=True)

evaluator.fit(X_train, y_train)

ModelEvaluator(base_multitarget=True,
               metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=['Alcohol',
                                                                  'Amphet',
                                                                  'Amyl',
                                                                  'Benzos',
                                                                  'Caff',
                                                                  'Cannabis',
                                                                  'Choc',
                                                                  'Coke',
                                                                  'Crack'],
                                            columns_to_delete=[],
                                            model=MultiOutputClassifier(estimator=LGBMClassifier(verbose=-1))))

In [13]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,accuracy,precision,recall,f1_macro,Targets
0,0.60424,0.325147,0.286855,0.296402,Ecstasy
1,0.839223,0.212204,0.174114,0.180049,Heroin
2,0.786219,0.28183,0.195527,0.208247,Ketamine
3,0.581272,0.215556,0.213898,0.205683,Legalh
4,0.639576,0.329797,0.311369,0.315952,LSD
5,0.742049,0.322678,0.199916,0.211242,Meth
6,0.584806,0.300544,0.30167,0.296532,Mushrooms
7,0.429329,0.283796,0.262817,0.258403,Nicotine
8,0.9947,0.331567,0.333333,0.332448,Semer
9,0.750883,0.194413,0.171523,0.172594,VSA
